In [1]:
from datasets import load_dataset

ds = load_dataset("keremberke/license-plate-object-detection", name="full")

c:\Users\Granth Bagadia\miniconda3\envs\aivid\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
for i in range(len(ds['train'])):
    example = ds['train'][i]
    first_image = example['image']
    first_boxes = example['objects']['bbox'][0]
    first_boxes[2] = first_boxes[2] + first_boxes[0]
    first_boxes[3] = first_boxes[3] + first_boxes[1]
    new_crop = first_image.crop(first_boxes)
    new_crop.save(f"train/{i}.jpg")

In [ ]:
import os
import cv2
import numpy as np
import albumentations as A
from tqdm import tqdm

In [ ]:
transform = A.Compose([
    A.GaussianBlur(blur_limit=(3, 7), p=0.5),        # Apply Gaussian Blur
    A.MotionBlur(blur_limit=(3, 7), p=0.5),          # Apply Motion Blur
    A.MedianBlur(blur_limit=3, p=0.5),               # Apply Median Blur
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),     # Add Gaussian Noise
    A.RandomBrightnessContrast(p=0.5)                # Random Brightness/Contrast
])

In [ ]:
def augment_images(input_dir, output_dir, num_augmentations=5):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for img_name in tqdm(os.listdir(input_dir)):
        img_path = os.path.join(input_dir, img_name)
        image = cv2.imread(img_path)
        if image is None:
            continue

        for i in range(num_augmentations):
            augmented = transform(image=image)
            augmented_image = augmented['image']

            # Save augmented image
            new_img_name = f"{os.path.splitext(img_name)[0]}_aug_{i}.jpg"
            cv2.imwrite(os.path.join(output_dir, new_img_name), augmented_image)


In [ ]:
input_directory = 'train_not_blur'
output_directory = 'train_blur'

augment_images(input_directory, output_directory)